In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [3]:
# Define the desired data structure
class Suggestions(BaseModel):
  words: List[str] = Field(description="list of substitue words based on context")
  
  # Throw error in case of receiving a numbered-list from API
  @validator('words')
  def not_start_with_number(cls, field):
    for item in field:
      if item[0].isnumeric():
        raise ValueError("The word can not start with a number")
    return field

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [4]:
from langchain.prompts import PromptTemplate

In [5]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

In [6]:
prompt = PromptTemplate(
  template=template,
  input_variables=['target_word', 'context'],
  partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [7]:
model_input = prompt.format_prompt(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson"
)

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_openai.llms import OpenAI

model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)

In [11]:
print(model_input.to_string())


Offer a list of suggestions to substitue the specified target_word based the presented context.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"words": {"title": "Words", "description": "list of substitue words based on context", "type": "array", "items": {"type": "string"}}}, "required": ["words"]}
```
target_word=behaviour
context=The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson



In [13]:
output = model.invoke(model_input.to_string())

In [14]:
print(output)


{"words": ["conduct", "manage", "handle", "oversee", "supervise"]}


In [15]:
parsed_output = parser.parse(output)

In [16]:
parsed_output

Suggestions(words=['conduct', 'manage', 'handle', 'oversee', 'supervise'])

In [17]:
parsed_output.words

['conduct', 'manage', 'handle', 'oversee', 'supervise']

In [ ]:
# Managing multiple outputs
template="""
Offer a list of suggestions to substitute the specified target_word based on the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

In [18]:
class Suggestions(BaseModel):
  words: List[str] = Field(description="list of substitue words based on context")
  reasons: List[str] = Field(description="the reasoning of why this word fits the context")
  
  @validator("words")
  def not_start_with_number(cls, field):
      for item in field:
        if item[0].isnumeric():
          raise ValueError("The word can not start with numbers!")
      return field
  
  @validator("reasons")
  def end_with_dot(cls, field):
      for idx, item in enumerate( field ):
        if item[-1] != ".":
          field[idx] += "."
      return field
    
    

In [22]:
parser = PydanticOutputParser(pydantic_object=Suggestions)
prompt = PromptTemplate(
  template=template,
  input_variables=['target_word', 'context'],
  partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [23]:
model_input = prompt.format_prompt(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson"
)

In [24]:
print(model_input.to_string())


Offer a list of suggestions to substitue the specified target_word based the presented context.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"words": {"title": "Words", "description": "list of substitue words based on context", "type": "array", "items": {"type": "string"}}, "reasons": {"title": "Reasons", "description": "the reasoning of why this word fits the context", "type": "array", "items": {"type": "string"}}}, "required": ["words", "reasons"]}
```
target_word=behaviour
context=The behaviour of the students in the classroom was disruptive and made it difficult for 

In [25]:
output = model.invoke(model_input.to_string())

In [26]:
print(output)


{"words": ["conduct", "manage", "handle", "oversee", "facilitate"], "reasons": ["These words all suggest a similar meaning to "behaviour" in the context of managing or controlling a situation, such as a classroom environment."]}




In [27]:
# Structured Output Parser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
  ResponseSchema(name="words", description="A substitue word based on context"),
  ResponseSchema(name="reasons", description="the reasoning of why this word fits the context.")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [30]:
prompt = PromptTemplate(
  template=template,
  input_variables=['target_word', 'context'],
  partial_variables={"format_instructions": parser.get_format_instructions()}
)
model_input = prompt.format_prompt(
  target_word="behaviour",
  context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson"
)
print(model_input.to_string())


Offer a list of suggestions to substitue the specified target_word based the presented context.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"words": string  // A substitue word based on context
	"reasons": string  // the reasoning of why this word fits the context.
}
```
target_word=behaviour
context=The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson



In [31]:
output = model.invoke(model_input.to_string())
print(output)

```json
{
	"words": "conduct",
	"reasons": "This word is a synonym for behaviour and fits the context of the teacher trying to manage the students' actions in the classroom."
},
{
	"words": "demeanor",
	"reasons": "This word refers to the outward behavior or conduct of a person and fits the context of the students' actions in the classroom."
},
{
	"words": "manners",
	"reasons": "This word refers to the way in which someone behaves or conducts themselves, and fits the context of the students' actions in the classroom."
},
{
	"words": "actions",
	"reasons": "This word is a synonym for behaviour and fits the context of the students' actions in the classroom."
},
{
	"words": "attitude",
	"reasons": "This word refers to the way in which someone behaves or conducts themselves, and fits the context of the students' actions in the classroom."
}
